<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/BarraQuelca_GuidoAlberto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se multiplicara dos matrices A y B, el resultado de la multiplicacion se guardara en la matriz C.
La multiplicacion de la matrices tiene muchas aplicaciones, una de ellas es en la generacion del movimientos de un objeto en los videojuegos como por ejemplo, la rotacion, el giro o la traslacion se los puede representar mediante una matriz. Al multiplicar algunas de las matrices mensionadas con la matriz de posicion del objeto, se obtiene el movimiento del mismo. 

El desarrollo teorico de la multiplicacion de matrices se mostrara a continuacion.

C = A.B
\begin{pmatrix}a_{11}&\cdots &a_{1n}\\\vdots &\ddots &\vdots \\a_{m1}&\cdots &a_{mn}\end{pmatrix}\begin{pmatrix}b_{11}&\cdots &b_{1p}\\\vdots &\ddots &\vdots \\b_{n1}&\cdots &b_{np}\end{pmatrix}\begin{pmatrix}a_{11}b_{11}+\cdots +a_{1n}b_{n1}&\cdots &a_{11}b_{1p}+\cdots +a_{1n}b_{np}\\\vdots &\ddots &\vdots \\a_{m1}b_{11}+\cdots +a_{mn}b_{n1}&\cdots &a_{m1}b_{1p}+\cdots +a_{mn}b_{np}\end{pmatrix}

La multiplicacion de matrices se puede realizar si la dimensiones de las matrices coinciden, por ejemplo una matriz A de dimension mxn y una matriz B de dimension nxp, se puede realizar la operacion de multiplicacion porque sus dimensiones coinciden, la cantidad de columnas de la matriz A es n y la cantidad de filas de la matriz B es n, la matriz del resultado C tiene la dimension de mxp. De forma simplificada la multiplicacion de matrices se realizas mediante la multiplicacion de los elementos de las filas de la matriz A con los elementos de las columnas de la matriz B, se suma cada multiplicacion y el resultado es un elemento de la matriz C,  debido a esto se valida las dimensiones de las matrices para su multiplicacion.


Para realizar la multipliacion se utilizara la siguiente ecuacion, la cual sale del desarrollo teorico.
<center>$C[f][c]=A[f][0]xB[0][c] + ....... +A[f][k]xB[k][c]$</center>
cuando k sea:
<center>$ k=cantColumnasMatrizA=cantFilasMatrizB$</center>

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y el manejo de la operacion de matriz a bajo nivel. El ejemplo es ilustrativo para entededer los multi hilos de la GPU en dos dimenciones, basicamente cada hilo realiza la misma operacion de multiplicar las filas por las columnas y el resultado se guarda en la matriz C.

---
# 2 Armado del ambiente
Toma las dimesiones de las matrices A Y B, la deja disponible al contexto de ejecuciòn del cuaderno colab.

In [1]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimensiones de la matrices:

try: 
  fila_mat_a =  1#@param {type:"integer"}
  colum_mat_a =  3#@param {type:"integer"}

  fila_mat_b =  3#@param {type:"integer"}
  colum_mat_b =  1#@param {type:"integer"}

except:
  print("Error de ingresar los parametros")








---
# 2.2 validar los parametros

In [2]:
#A.B
if fila_mat_a < 0 or colum_mat_a < 0:
  print("ERROR LAS DIMENCIONES DE LA MATRIZ A TIENE QUE SER MAYOR A 0")
elif fila_mat_b < 0 or colum_mat_b < 0:
  print("ERROR LAS DIMENCIONES DE LA MATRIZ B TIENE QUE SER MAYOR A 0")
elif colum_mat_a != fila_mat_b:
  print("ERROR LAS DIMENCIONES DE LAS MATRIZ NO PERMITEN REALIZAR LA MULTIPLICACION")
else: 
  print("OK LAS MATRICES SE PUEDEN MULTIPLICAR")

OK LAS MATRICES SE PUEDEN MULTIPLICAR


---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [3]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 6.9MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620990 sha256=784be6b24a78e7fcdae8b38e99471cd51f0b106868963cfb2ca1a20943e4dcbf
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=1d338249e864ffb7c5d2be67840232ab9ce33f2e46d84824b6e94cb7ab05fc89
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


---
# 3 Desarrollo
Ejecución del algoritmo de multiplicacion de matrices secuencial y paralelo.

---
# 3.1 Desarrollo Secuencial
Ejecución del algoritmo de multiplicacion de matrices en forma secuencial utilizando la CPU.

In [4]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  import numpy
  
  
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  
  tiempo_definicion_matrices = datetime.now()
  
  # CPU - Defino la memoria de la matriz A en cpu.
  matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
  matriz_a = matriz_a.astype(numpy.int32())
  
  # CPU - Defino la memoria de la matriz B en cpu.
  matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
  matriz_b = matriz_b.astype(numpy.int32())
  
  # CPU - Defino la memoria de la matriz resultado C en cpu.
  matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c = matriz_c.astype(numpy.int32())
  matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())
  
  tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices
  
  tiempo_multiplicacion_secuencial = datetime.now()
  
  for f in range(fila_mat_a):
    for c in range(colum_mat_b):
      for inter in range(colum_mat_a):
        matriz_c_secuencial[f][c] += matriz_a[f][inter]*matriz_b[inter][c]
  
  tiempo_multiplicacion_secuencial = datetime.now() - tiempo_multiplicacion_secuencial
  tiempo_total = datetime.now() - tiempo_total


  print("Matiz A:\n", matriz_a)
  print("Matiz B:\n", matriz_b)
  print("Matiz C:\n", matriz_c_secuencial)
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo CPU  : ", tiempo_en_ms( tiempo_multiplicacion_secuencial ), "[ms]" )
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

Matiz A:
 [[-7 -4 -3]]
Matiz B:
 [[ -6]
 [-19]
 [  0]]
Matiz C:
 [[118]]
Tiempo TOTAL:  0.532 [ms]
Tiempo CPU  :  0.018 [ms]


---
# 3.2 Desarrollo Paralelo
Ejecución del algoritmo de multiplicacion de matrices en forma paralela utilizando la GPU.

In [5]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()

  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------

  tiempo_definicion_matrices = datetime.now()

  # CPU - Defino la memoria de la matriz A en cpu.
  matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
  matriz_a = matriz_a.astype(numpy.int32())

  # CPU - Defino la memoria de la matriz B en cpu.
  matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
  matriz_b = matriz_b.astype(numpy.int32())

  # CPU - Defino la memoria de la matriz resultado C en cpu.
  matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c = matriz_c.astype(numpy.int32())
  matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())

  tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices

  tiempo_reserva_memoria_GPU = datetime.now()
  # CPU - reservo la memoria GPU.
  matriz_a_Gpu = cuda.mem_alloc(matriz_a.nbytes)
  matriz_b_Gpu = cuda.mem_alloc(matriz_b.nbytes)
  matriz_c_Gpu = cuda.mem_alloc(matriz_c.nbytes)
  tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU

  tiempo_copia_memoria_GPU = datetime.now()
  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(matriz_a_Gpu, matriz_a)
  cuda.memcpy_htod(matriz_b_Gpu, matriz_b)
  cuda.memcpy_htod(matriz_c_Gpu, matriz_c)
  tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void multiplicar(int ancho, int alto, int inter, int *matriz_a , int *matriz_b, int *matriz_c)
  {
      // Calculo las coordenadas del Thread en dos dimensiones.
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      int idy = threadIdx.y + blockIdx.y*blockDim.y;

      // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
      if( idx < ancho && idy < alto ) {
        int indice = idy+(idx*ancho);
        int i = 0;
        int resul = 0;

        //
        while (i<inter) {
          //multiplico fila por columna y sumo el resultado de cada componente
          resul += matriz_a[idx*inter + i]*matriz_b[idy + i*inter];
          i++;
        }
        //paso el resultado obtenido a la componente de la matriz
        matriz_c[indice] = resul;
      }
  }

  """)

  # CPU - Genero la función kernel.
  kernel = module.get_function("multiplicar")

  dim_hilo_x = 16
  dim_bloque_x = numpy.int( (fila_mat_a+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 19
  dim_bloque_y = numpy.int( (colum_mat_b+dim_hilo_y-1) / dim_hilo_y )

  print( "Thread: [",
        dim_hilo_x,
        ",",
        dim_hilo_y,
        " ], Bloque : [",
        dim_bloque_x,
        ",",
        dim_bloque_y,
        "]" )
  print( "Total de Thread: [",
        dim_hilo_x*dim_bloque_x,
        ",",
        dim_hilo_y*dim_bloque_y,
        " ]",
        " = ",
        dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

  tiempo_multiplicacion_paralela = datetime.now()

  kernel( numpy.int32(fila_mat_a),
          numpy.int32(colum_mat_b),
          numpy.int32(colum_mat_a),
          matriz_a_Gpu,
          matriz_b_Gpu,
          matriz_c_Gpu,
          block=( dim_hilo_x, dim_hilo_y, 1 ),
          grid=(dim_bloque_x, dim_bloque_y,1) )

  tiempo_multiplicacion_paralela = datetime.now() - tiempo_multiplicacion_paralela

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( matriz_c, matriz_c_Gpu )
  tiempo_total = datetime.now() - tiempo_total

  print("Matiz A:\n", matriz_a)
  print("Matiz B:\n", matriz_b)
  print("Matiz C:\n", matriz_c)
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_multiplicacion_paralela ), "[ms]" )

except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

Thread: [ 16 , 19  ], Bloque : [ 1 , 1 ]
Total de Thread: [ 16 , 19  ]  =  304
Matiz A:
 [[ 5 -4 -3]]
Matiz B:
 [[-1]
 [-5]
 [-6]]
Matiz C:
 [[-5]]
Tiempo TOTAL:  1137.835 [ms]
Tiempo GPU  :  1.769 [ms]


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random(n)       | Crear una vector con numeros ramdoms de n elementos.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las imagenes en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las imagenes al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x            | Calcula las dimensiones para la ejecuciòn de 1D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa los atributos de la imagen.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.

---
# 5 Conclusiones

Las conclusiones son explicadas en clase...

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[4] 2009, SINTESIS DIGITAL DE COLOR UTILIZANDO TONOS DE GRIS, ING. JESÚS GUSTAVO FLORES ERAÑA : [PDF](https://ninive.uaslp.mx/xmlui/bitstream/handle/i/2264/MCA1SDC00901.pdf?sequence=1&isAllowed=y)
